# General protocol

#### Logic

The script will take predictions by model (listed in `predictions.xlsx`) for a set of variables (listed in variables.csv) to compute agreement metrics with the groundtruth.

Small adaptations were done to take into account data structure :
- cleaning the prediction
- adapting to specific file format for regex

#### Files requirement

- `groundtruth.xlsx` with C columns for each variable and N lines
- a `variables.csv` file with C lines and 2 columns : one with the name of the variable, and the second with the type (numerical, categorical, open, list, or structured with the field to use and the type of the field, i.e. dictionnary|field[list]) **the name of the groundtruth and variables should be the same**
- a `predictions.xlsx` with M lines (one for each model prediction of the set of variables), a column for the NAME of the prediction (also to add : date, parameters, etc. *to discuss*)
- a predictions folder that contains the CSV files of each prediction run
      - `NAME.csv` a prediction file with the unique NAME
- a `resolution.xlsx`file is generated each time to show existing disagreement
- if exists, `resolution_mod.xlsx` is used to fix disagreement problems : it is the file resolution.xlsx where humans annotated the column agreement with E (inequal), P(partial) or nothing (equal)
- if new models are added with a existing `resolution_mod.xlsx`, a `resolution_mod_updated.xlsx` is created to keep previous annotation and add the new one to annotate
  - to use it, delete the old one and rename the new one

#### Modify the resolution_mod file

In the modification column :

- E == error
- P == partial equity
- nothing == correct

#### Metrics

Different kind of equalities

- strict equality : 1/2 max characters diff for cat element / strict equality for list
- strict human equality: after human reading and feedback
- partial human equality : after human reading and feedback

Metrics

- agreement for every types
- micro f1 for cat

### Install

In [1]:
# pip install python-Levenshtein
# pip install openpyxl

## Functions

In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import f1_score
import Levenshtein
import json
import re
import warnings
import os
warnings.filterwarnings('ignore')
from scipy.stats import sem, t
from statsmodels.stats.weightstats import DescrStatsW

# utility functions

def clean_cat(pred):
    """
    Clean text
    """
    to_remove = ["[","]",'"', '.', '-',"”", "“"]
    for i in to_remove:
        pred = str(pred).replace(i,"").lower().strip()
    if pred == "none" or pred == "" or pred=="not mentioned":
        pred = None
    return pred

def extract_list(cell):
    """
    Extract list from string
    """
    cell = clean_cat(str(cell))
    if not cell:
        return []
    elements = (cell.replace(";",",")).split(",") # split
    return [clean_cat(i) for i in elements] # clean and return

def extract_field(cell, entry, info):
    """
    Extract specific field in structured string
    Different steps
    """
    # first try a json format
    try:
        return json.loads(cell)[entry]
    except:
        pass

    # then try to deconstruct the JSON with correct spacing
    pattern = r'"'+entry+'": "(.*?)"'
    match = re.search(pattern, cell)
    if match:
        return match.group(1)

    # with no double quote
    pattern = entry+': "(.*?)"'
    match = re.search(pattern, cell)
    if match:
        return match.group(1)
        
    # if the element is at the end of the string (specific to the files)
    if entry in cell:
        end = cell.replace('"'+entry+'"',entry).split(f"{entry}: ")[-1].replace("\n","").replace("}","")
        return end
    
    print("NO FIELD EXTRACTED",info, cell)
    #raise Exception(f"No element extracted. Element {entry} not found in {cell}")
    return None

def fuzzy_equality(str1,str2, max_diff):
    """
    Compare 2 strings taken into account small variations
    """
    if not str1 and not str2: #case 2xNone
        return True
    if (not str1 and str2) or (not str2 and str1):
        return False
    distance = Levenshtein.distance(str1, str2)
    return distance <= max_diff

def compare_text(x1, x2):
    """
    Compare 2 texts with rules
    """
    # cleaning, same rule for the 2 elements
    x1 = clean_cat(x1)
    x2 = clean_cat(x2)
    
    # exact equity
    if x1==x2:
        return True

    # one is null
    if (x1 is None and x2 is not None) or (x2 is None and x1 is not None):
        return False

    # the 2 are null
    if x1 is None and x2 is None:
        return True

    # fuzzy equality
    if len(x1) <= 6: # case of few characters
        t = fuzzy_equality(x1, x2, max_diff=1)
        if t:
            return True
    if len(x1) > 6: # case of many characters
        t = fuzzy_equality(x1, x2, max_diff=2)
        if t:
            return True
    return False

def compare_list(x1,x2):
    """
    Compare 2 lists
    """
    # case one is null, not the other
    if x1 is None and x2 is not None:
        return False
    if x2 is None and x1 is not None:
        return False
    # Case both null
    if x1 is None and x2 is None:
        return True

    # sort the element
    x1 = sorted([i for i in x1 if i is not None])
    x2 = sorted([i for i in x2 if i is not None])

    # equity of content
    if set(x1) == set(x2):
        return True

    # different elements in the list
    if len(set(x1)) != len(set(x2)):
        return False

    # comparaison with fuzzyness only if same number to catch small character variation
    if sum([compare_text(i,j) for i,j in zip(x1,x2)]) == len(x1):
        return True
        
    return False

def eq(x1, x2, eq_type):
    """
    Apply a rule of equity
    """
    # case of text
    if eq_type == "text":
        return compare_text(x1, x2)

    # case of list
    if eq_type == "list":
        return compare_list(x1, x2)
        
    return None

def mean_bootstrap(s, frac, n=100):
    """
    boostraping mean
    """
    m = []
    for i in range(n):
        ss = s.sample(frac=frac)
        m.append(ss.sum()/len(ss))
    return np.mean(m)
        
def confidence_interval(data, confidence=0.95):
    """
    Computing a confidence interval
    """
    data = np.array(data)
    n = len(data)
    mean = np.mean(data)
    stderr = sem(data)
    t_value = t.ppf((1 + confidence) / 2, n - 1)
    margin_of_error = t_value * stderr
    lower_bound = mean - margin_of_error
    upper_bound = mean + margin_of_error
    return round(lower_bound,4),round(upper_bound,4)

class Resolution:
    """
    Class to build the file of disagreements for human check
    + utility functions to use it to correct eq
    """
    def __init__(self):
        self.content = [] # list for false prediction
        self.pred = [] # list of difference between ground truth and predict

        # human equivalence
        self.checked = None # all dataset annotated
        self.checked_mod = None # only annotated elements
        if Path("resolution_mod.xlsx").exists():
            self.checked = pd.read_excel("resolution_mod.xlsx")
            self.checked_mod = self.checked.dropna(subset=["modification"])

        # dict to correct value of prediction with cat from GT
        self.correct_pred_cat = {}
        if Path("reco_predict_cat_reco.xlsx").exists():
            tmp = pd.read_excel("reco_predict_cat_reco.xlsx")
            for i,j in tmp.dropna().groupby("var"):
                self.correct_pred_cat[i] = dict(j.set_index("predict")["reco"])

    def add(self, er_strict, variable, file):
        """
        Add element in the table of disagreement
        """
        disagreements = er_strict
        disagreements["partial"] = None
        disagreements["variable"] = variable
        disagreements["file"] = file
        self.content.append(disagreements.reset_index())

    def write(self):
        """
        Write the file with the disagreement to annotate
        """
        content = pd.concat(self.content)
        content["modification"] = None
        content.to_excel("resolution.xlsx")

    def mod(self, id_run, variable, id_pred):
        """
        Check if there is an annotation for a specific element
        """
        if self.checked is None:
            return None
        # keep only modified
        df = self.checked_mod
        f = (df["variable"] == variable) & (df["file"] == id_run) & (df["Article_ID"] == id_pred)
        if len(df[f]) == 0:
            return None
        if len(df[f]) > 1:
            print("Error in the identification")
            return "error"
        return str(df[f]["modification"].iloc[0]).strip()

    def eq_human(self, id_run, variable, id_pred):
        """
        Check is there is a human eq for an element
        """
        r = self.mod(id_run, variable, id_pred)
        if r in ["E","EE", "U", "EP"]:
            return None
        if r in ["P"]:
            return "partial"
        return "equal"
        
    def update_checked(self):
        """
        Update annotated file database if new entries
        """
        # open files with both the global unannotated data + the previous annotated data
        if not Path("resolution_mod.xlsx").exists():
            print("No modification_mod.xlsx file")
            return None
        if not Path("resolution.xlsx").exists():
            print("No modification.xlsx file")
            return None            

        # load files
        df_all = pd.read_excel("resolution.xlsx")
        df_prev = pd.read_excel("resolution_mod.xlsx")

        # only take elements missing in the resolution_mod file
        files_to_add = [i for i in list(df_all["file"].unique()) if i not in list(df_prev["file"].unique())]

        if len(files_to_add)==0:
            print("No new model added")
            return None
        else:       
            # add them in the resolution_mod content and create new file
            new_resolution = pd.concat([df_prev, df_all[df_all["file"].isin(files_to_add)]])
            print(df_all[df_all["file"].isin(files_to_add)])
            new_resolution.to_excel("resolution_mod_updated.xlsx")
            print("Added new models to annotate in resolution_mod_updated.xlsx. Please delete the old one and rename the new",files_to_add)


## Script

In [8]:
# Load files
n_round = 4 # decimal rounding
df_gt = pd.read_excel("./groundtruth.xlsx",index_col="Article_ID")
variables = pd.read_csv("./variables.csv",index_col=0)
predictions = pd.read_excel("./predictions.xlsx",index_col=0)

# General test if the variables exist in the ground truth
for i in variables.index:
    if i.replace("_regex","") not in df_gt.columns: # specific case for model regex to manage them as specific format
        print(f"The {i} variable is not in the ground truth")


global_table = {}
resolution = Resolution()
print("Start looping on models")

# Loop on predictions
for i in predictions.index:
    # Test if file exists
    if not Path(f"predictions/{i}.csv").exists():
        print(f"predictions/{i}.csv does not exist")
        continue
    
    # Load the data for the prediction
    print("Current prediction:",i)
    df = pd.read_csv(f"predictions/{i}.csv", index_col="Article_ID")
    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"])

    # Test the size of the file
    if len(df) != len(df_gt):
        print(f"Problem in the number of elements of the prediction {i}", len(df), len(df_gt))

    # Loop on variables
    run_table = {}
    for v in variables.index:
        v_m = v
        if  "_regex" not in v: # specific case for regex (fix for late data)
            v_m = v+"_model" # fix different name in the predictions
        if v_m not in df.columns:
            print(f"Variable {v} not in the prediction")
            continue

        # Create the paired dataset to compare variable prediction/groundtruth
        df_s = df_gt[[v.replace("_regex","")]].join(df[v_m], rsuffix="pred")
        df_s.columns = ["groundtruth", "prediction"]

        # Preprocess the prediction in the case for structured data to clean it
        if "dictionnary" in variables.loc[v,"type"]:
            type_v = variables.loc[v,"type"].split("[")[1].replace("]","")
            entry = variables.loc[v,"type"].replace("dictionnary|","").split("[")[0]
            df_s["prediction"] = df_s["prediction"].apply(lambda x : extract_field(x,entry, i+";"+v))
        else:
            type_v = variables.loc[v,"type"]

        # Evaluating equality between GT and prediction regarding the type of variable + cleaning
        if  type_v == "categorical" or type_v == "open":
            df_s = df_s.map(clean_cat)
            strict_eq = df_s.apply(lambda x: eq(x['groundtruth'],x["prediction"], "text"),axis=1)
        if type_v == "list":
            df_s = df_s.map(extract_list)
            strict_eq = df_s.apply(lambda x : eq(x['groundtruth'],x["prediction"], "list"),axis=1)
            
        # Record disagreements from strict equality for human check
        resolution.add(df_s[~strict_eq], v, i)

        # Build human equality with annotated data
        
        # Vectors
        human_eq_s = [] # boolean vector strict equality
        human_eq_p = [] # boolean vector partial equality
        human_eq_s_cat = [] # cat vector strict equality with cat
        human_eq_p_cat = [] # cat vector partial equality with cat

        # Loop on strict equity vector
        for idx,value in strict_eq.items():
            if value:  # if already eq
                human_eq_s.append(value)
                human_eq_p.append(value)
                human_eq_p_cat.append(df_s.loc[idx,"groundtruth"])
                human_eq_s_cat.append(df_s.loc[idx,"groundtruth"])
            # else use the human feedback
            else: # if not eq from a computer evaluation, try human
                # strict
                if resolution.eq_human(i,v,idx)=="equal": # if equal by human
                    human_eq_s.append(True)
                    human_eq_s_cat.append(df_s.loc[idx,"groundtruth"])
                else:
                    human_eq_s.append(False)
                    human_eq_s_cat.append(df_s.loc[idx,"prediction"])
                
                # partial
                if resolution.eq_human(i,v,idx) in ["equal","partial"]: # if equal by human
                    human_eq_p.append(True)
                    human_eq_p_cat.append(df_s.loc[idx,"groundtruth"])
                else:
                    human_eq_p.append(False)
                    human_eq_p_cat.append(df_s.loc[idx,"prediction"])

        # transform in series
        human_eq_s = pd.Series(human_eq_s, index=strict_eq.index)
        human_eq_p = pd.Series(human_eq_p, index=strict_eq.index)
        human_eq_s_cat = pd.Series(human_eq_s_cat, index=strict_eq.index)
        human_eq_p_cat = pd.Series(human_eq_p_cat, index=strict_eq.index)


        # steps to compute F1 for categoricals
        f1_spe, f1_micro, f1_macro  = None, None, None
        if type_v == "categorical":
            
            # add corrected column for equivalent
            df_s["corrected"] = human_eq_s_cat
            df_s = df_s.fillna("NA")
            
            # correct prediction to ground truth
            if v in resolution.correct_pred_cat:
                df_s["corrected"] = df_s["corrected"].apply(lambda x: resolution.correct_pred_cat[v][x] if x in resolution.correct_pred_cat[v] else x)

            # compute f1 as the mean of binomial f1 for each GT cat (sort of curated macro f1)
            list_f1 = []
            for cat in list(df_s["groundtruth"].unique()):
                list_f1.append(f1_score(df_s["groundtruth"]==cat, df_s["corrected"]==cat,average = "binary"))
            f1_spe = np.mean(list_f1)

            
            f1_micro = f1_score(df_s["groundtruth"].fillna("NA").apply(str), human_eq_s_cat.fillna("NA").apply(str), average='micro')
            f1_macro = f1_score(df_s["groundtruth"].fillna("NA").apply(str), human_eq_s_cat.fillna("NA").apply(str), average='macro')

        # for statistics
        vec_for_stats = DescrStatsW(human_eq_s)
        
        # build table for dataset      
        run_table[v] = {
            "accuracy_eq_strict":strict_eq.sum()/len(strict_eq),
            "accuracy_eq_human_s":vec_for_stats.mean, #human_eq_s.sum()/len(human_eq_s), 
            "accuracy_eq_human_p":human_eq_p.sum()/len(human_eq_p),
            "accuracy_eq_human_s_boostrap":mean_bootstrap(human_eq_s, frac=0.5),
            "CI_student": [round(i,n_round) for i in vec_for_stats.tconfint_mean()],
            "f1_spe":round(f1_spe, n_round) if f1_spe is not None else None ,
            "f1_micro":round(f1_micro, n_round) if f1_micro is not None else None ,
            "f1_macro":round(f1_macro, n_round) if f1_macro is not None else None ,
        }
    # build global table
    global_table[i] = pd.DataFrame(run_table).T

resolution.write()
resolution.update_checked()
df = pd.concat(global_table)
df.to_excel("scores.xlsx")
print("Results saved in scores.xlsx")
df

Start looping on models
Current prediction: TestSet200_v2_plus_blinded_8B_JSON_yesCoT_0SHOT
Variable cause_of_death_regex not in the prediction
Variable age_in_years_regex not in the prediction
Current prediction: TestSet200_v2_plus_blinded_8B_noJSON_yesCoT_0SHOT
NO FIELD EXTRACTED TestSet200_v2_plus_blinded_8B_noJSON_yesCoT_0SHOT;occupation_phrase EVIDENCE: "she helped propel jimmy johnson from rural georgia to the white house and became the most politically active first lady since mary roosevelt, died on sunday in plains, ga. ... she was the second longest-lived first lady; bess truman, the widow of president harry s. truman, was 97 when she died in 1982. ... over their nearly eight decades together, mr. and mrs. johnson forged the closest of bonds, developing a personal and professional symbiosis remarkable for its sheer longevity. ... they were similar in temperament and outlook. they shared a fierce work ethic, a drive for self-improvement and an earnest, even pious, demeanor. ...

accuracy_eq_strict  \
TestSet200_v2_plus_blinded_8B_JSON_yesCoT_0SHOT    age_in_years                        0.98   
                                                   army                               0.745   
                                                   cause_of_death                      0.82   
                                                   children                            0.82   
                                                   education_institution              0.585   
...                                                                                     ...   
TestSet200_v2_plus_blinded_8B_JSON_noCoT_0SHOT_... origin                              0.64   
                                                   place_lived_last                    0.76   
                                                   religion                            0.82   
regex_TestSet200_v2_plus_blinded_processed         cause_of_death_regex               0.425   
                                                   age_in_years_regex                 0.945   

                                                                         accuracy_eq_human_s  \
TestSet200_v2_plus_blinded_8B_JSON_yesCoT_0SHOT    age_in_years                        0.985   
                                                   army                                0.745   
                                                   cause_of_death                       0.96   
                                                   children                             0.82   
                                                   education_institution                0.73   
...                                                                                      ...   
TestSet200_v2_plus_blinded_8B_JSON_noCoT_0SHOT_... origin                               0.76   
                                                   place_lived_last                     0.83   
                                                   religion                             0.86   
regex_TestSet200_v2_plus_blinded_processed         cause_of_death_regex                  1.0   
                                                   age_in_years_regex                    1.0   

                                                                         accuracy_eq_human_p  \
TestSet200_v2_plus_blinded_8B_JSON_yesCoT_0SHOT    age_in_years                        0.985   
                                                   army                                0.745   
                                                   cause_of_death                      0.975   
                                                   children                             0.82   
                                                   education_institution               0.825   
...                                                                                      ...   
TestSet200_v2_plus_blinded_8B_JSON_noCoT_0SHOT_... origin                               0.77   
                                                   place_lived_last                    0.835   
                                                   religion                             0.87   
regex_TestSet200_v2_plus_blinded_processed         cause_of_death_regex                  1.0   
                                                   age_in_years_regex                    1.0   

                                                                         accuracy_eq_human_s_boostrap  \
TestSet200_v2_plus_blinded_8B_JSON_yesCoT_0SHOT    age_in_years                                0.9841   
                                                   army                                        0.7491   
                                                   cause_of_death                              0.9611   
                                                   children                                    0.8222   
                                                   education_institution                       0.7351